In [102]:
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd

load_dotenv()

dbname=os.getenv("DB_NAME"),

print(dbname)
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    print("Connection established successfully!")
except Exception as e:
    print("Connection failed:", e)


cursor = conn.cursor()

cursor.execute("SELECT * FROM potholes")

# Fetch results (if applicable)
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
cursor.close()
conn.close()

('HotPot',)
Connection established successfully!


In [103]:
#Changing the first_reported_date column to just date
for index, row in df.iterrows():
    df['first_reported_date'] =  pd.to_datetime(df['first_reported_date']).dt.date

In [104]:
#df with just the pothole_id, pothole_size and first_reported_date
df_one = df[['pothole_id', 'pothole_size', 'first_reported_date']]
df_one


,pothole_id,pothole_size,first_reported_date
0,40,Small,2024-06-19
1,41,Medium,2024-06-19
2,43,Large,2024-06-19
3,44,Small,2024-06-19
4,45,Small,2024-07-02
5,48,Large,2024-07-02
6,42,Medium,2024-06-19
7,53,Small,2024-07-03
8,54,Small,2024-07-03
9,55,Small,2024-07-03


In [105]:
#First response from Gemini with order of pothole_id
json_data = [
  {
    "pothole_id": 43
  },
  {
    "pothole_id": 48
  },
  {
    "pothole_id": 41
  },
  {
    "pothole_id": 42
  },
  {
    "pothole_id": 56
  },
  {
    "pothole_id": 40
  },
  {
    "pothole_id": 44
  },
  {
    "pothole_id": 45
  },
  {
    "pothole_id": 53
  },
  {
    "pothole_id": 54
  },
  {
    "pothole_id": 55
  }
]

### For part 2 getting the distances and times from the openRouteService API

##### Format for api is {"coordinates":[[8.681495,49.41461],[8.686507,49.41943],[8.687872,49.420318]]}

In [106]:
import json

data = json.loads(json.dumps(json_data))

order = []
for item in data:
    order.append(item['pothole_id'])

print(order)

#Sorting the df_twoPrep by the order using the order list as the index
df_twoPrep = df[['pothole_id', 'coordinates']]
df_twoPrep = df_twoPrep.set_index('pothole_id').loc[order].reset_index()
df_twoPrep

#iterating through the df_twoPrep and create a new list of coordinates, swapping the lat and long
coordinates = []
for index, row in df_twoPrep.iterrows():
    coordinates.append(row['coordinates'].split(','))
    coordinates[index] = [float(i) for i in coordinates[index]]
    coordinates[index] = [coordinates[index][1], coordinates[index][0]]
coordinates



[43, 48, 41, 42, 56, 40, 44, 45, 53, 54, 55]


[[-80.52067374289663, 43.478928936613585],
 [-80.51149, 43.41541],
 [-80.51565095313065, 43.47555811807502],
 [-80.51872032255609, 43.483713466146924],
 [-80.50617618622455, 43.492997054664215],
 [-80.51461819739562, 43.482921333260826],
 [-80.521015, 43.48088],
 [-80.51078069502678, 43.482236381246224],
 [-80.5098641632737, 43.49572546051195],
 [-80.5213569262076, 43.489874437090805],
 [-80.52306039824327, 43.47493797604383]]

In [107]:
ORS_API_KEY = os.getenv("ORS_KEY")

import requests

#set the headers and the payload
headers = {
    'Content-Type': 'application/json; charset=utf-8',
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ORS_API_KEY
}

payload = {
    'coordinates': coordinates
}

#make the request
response = requests.post('https://api.openrouteservice.org/v2/directions/driving-car/json', headers=headers, json=payload)
response.json()

distance = []
duration = []

for i in range(len(response.json()['routes'][0]['segments'])):
    distance.append(response.json()['routes'][0]['segments'][i]['distance'])
    duration.append(response.json()['routes'][0]['segments'][i]['duration'])

distance
duration

duration.append(0)
distance.append(0)




In [108]:
df_three = df_one
df_three = df_three.set_index('pothole_id').loc[order].reset_index()
df_three['distance (meters)'] = distance
df_three['duration (seconds)'] = duration
#df_three

df_three['distance (KM)'] = df_three['distance (meters)'] / 1000
df_three['duration (MINS)'] = df_three['duration (seconds)'] / 60
df_three

,pothole_id,pothole_size,first_reported_date,distance (meters),duration (seconds),distance (KM),duration (MINS)
0,43,Large,2024-06-19,15621.1,870.6,15.6211,14.510000
1,48,Large,2024-07-02,14741.0,898.6,14.7410,14.976667
2,41,Medium,2024-06-19,1416.5,135.3,1.4165,2.255000
3,42,Medium,2024-06-19,1934.7,230.2,1.9347,3.836667
4,56,Large,2024-07-04,3251.9,391.5,3.2519,6.525000
5,40,Small,2024-06-19,1532.4,203.1,1.5324,3.385000
6,44,Small,2024-06-19,1287.7,131.9,1.2877,2.198333
7,45,Small,2024-07-02,2814.4,294.8,2.8144,4.913333
8,53,Small,2024-07-03,1385.6,256.1,1.3856,4.268333
9,54,Small,2024-07-03,2118.2,267.3,2.1182,4.455000


In [109]:
df_json = df_three.to_json(orient='records')
df_json

'[{"pothole_id":43,"pothole_size":"Large","first_reported_date":1718755200000,"distance (meters)":15621.1,"duration (seconds)":870.6,"distance (KM)":15.6211,"duration (MINS)":14.51},{"pothole_id":48,"pothole_size":"Large","first_reported_date":1719878400000,"distance (meters)":14741.0,"duration (seconds)":898.6,"distance (KM)":14.741,"duration (MINS)":14.9766666667},{"pothole_id":41,"pothole_size":"Medium","first_reported_date":1718755200000,"distance (meters)":1416.5,"duration (seconds)":135.3,"distance (KM)":1.4165,"duration (MINS)":2.255},{"pothole_id":42,"pothole_size":"Medium","first_reported_date":1718755200000,"distance (meters)":1934.7,"duration (seconds)":230.2,"distance (KM)":1.9347,"duration (MINS)":3.8366666667},{"pothole_id":56,"pothole_size":"Large","first_reported_date":1720051200000,"distance (meters)":3251.9,"duration (seconds)":391.5,"distance (KM)":3.2519,"duration (MINS)":6.525},{"pothole_id":40,"pothole_size":"Small","first_reported_date":1718755200000,"distance (m